# Construyendo modelos con el API Funcional de Keras
Una capa es una transformación de entrada/salida
```
dense = keras.layers.Dense(units=16)
```
Un modelo es un grafo acíclico dirigido de capas. 

El API Functional es la forma más frecuente y poderosa de crear modelos con Keras. Para construir modelos con el API Functional, se especifican las dimensiones  y el tipo de datos de los datos de entrada. Es posible especificar dimensiones variables con None. Por ejemplo, una imagen 200x200 RGB tiene la forma (200,200, 3), paro una imagen de dimensiones arbitrarias RGB tendría la forma (None, None, 3).

```python
# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(None, None, 3))
```
Después de definir las entradas, es posible encadenar las transformaciones de las capas después de las entradas hasta la salida final.


In [13]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as k
import pathlib

In [8]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [9]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [16]:
#Obtener un dataset etiquetado de imágenes en disco
# Create a dataset.
image_dataset = k.preprocessing.image_dataset_from_directory(
  data_dir, batch_size=32, image_size=(180, 180))

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in image_dataset.take(1):
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32

Found 3670 files belonging to 5 classes.
(32, 180, 180, 3)
<dtype: 'float32'>
(32,)
<dtype: 'int32'>


In [17]:
image_dataset

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

In [18]:
from tensorflow.keras import layers

# Center-crop images to 150x150
x = k.layers.experimental.preprocessing.CenterCrop(height=150, width=150)(image_dataset)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

ValueError: Attempt to convert a value (<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>) to a Tensor.